<p style="font-weight:bold;"> <span style="font-size: 36px"> Test Practical Cases </span> </p>

# Csm Switch : Single vs Multiple

In [0]:
#!import "../PracticalUseCases/SingleVsMultipleCsmSwitch/CsmSwitchReports"

In [0]:
using FluentAssertions;

## Fulfillment Cashflows

In [0]:
var fcf = fulfillmentCashflows.GetDataCube().ToDictionaryGrouped(x => x.GroupOfContract, x => x.ToArray());

In [0]:
fcf.Keys.Count.Should().Be(2);
fcf[fcf.Keys.First()].All(x => Math.Abs(fcf[fcf.Keys.Last()].Single(y => y.VariableType == x.VariableType && y.Novelty == x.Novelty && 
                                                                   y.EstimateType == x.EstimateType && y.EconomicBasis == x.EconomicBasis &&
                                                                   y.AmountType == x.AmountType).Value - x.Value) < 10E-4)
                                                                   .Should().BeTrue();

## Actuals

In [0]:
var actuals = writtenActual.GetDataCube().ToDictionaryGrouped(x => x.GroupOfContract, x => x.ToArray());

In [0]:
actuals.Keys.Count.Should().Be(2);
actuals[actuals.Keys.First()].All(x => Math.Abs(actuals[actuals.Keys.Last()].Single(y => y.VariableType == x.VariableType && y.Novelty == x.Novelty && 
                                                                   y.EstimateType == x.EstimateType && y.EconomicBasis == x.EconomicBasis &&
                                                                   y.AmountType == x.AmountType).Value - x.Value) < 10E-4)
                                                                   .Should().BeTrue();

## Allocated Technical Margin

In [0]:
allocatedTechnicalMargins.GetDataCube().Select(x => x.GroupOfContract).Distinct().Count().Should().Be(2);

In [0]:
var eops = allocatedTechnicalMargins.GetDataCube().Where(x => x.VariableType == "EOP");

In [0]:
eops.Where(x => x.EstimateType == "C").Count().Should().Be(2);
eops.Where(x => x.EstimateType == "C").All(x => Math.Abs(x.Value - 5.71) < 0.01).Should().BeTrue();
eops.Single(x => x.EstimateType == "L").Value.Should().BeApproximately(0,Precision);

In [0]:
var lc = allocatedTechnicalMargins.GetDataCube().Where(x => x.EstimateType == "L");
var lcIa = lc.Single(x => x.VariableType == "IA").Value;

In [0]:
lc.All(x => x.Novelty != "I" && x.GroupOfContract == "MZ1.1").Should().BeTrue();

In [0]:
var lcCombined = lc.Where(x => x.Novelty == "C" && x.GroupOfContract == "MZ1.1");

In [0]:
lcCombined.Count().Should().Be(2);
lcCombined.Where(x => x.VariableType == "CL").Count().Should().Be(1);
lcCombined.Where(x => x.VariableType == "EOP").Count().Should().Be(1);

## Financial Performance

In [0]:
var mz11 = financialPerformance.GetDataCube().Where(x => x.GroupOfContract == "MZ1.1");
var mz12 = financialPerformance.GetDataCube().Where(x => x.GroupOfContract == "MZ1.2");

In [0]:
mz11.Sum(x => x.Value).Should().BeApproximately(mz12.Sum(x => x.Value), 10E-4);

In [0]:
mz11.Single(x => x.VariableType == "IR5" && x.EstimateType =="C").Value
    .Should().BeApproximately(
        mz12.Single(x => x.VariableType == "IR5" && x.EstimateType =="C").Value + lcIa,
        Precision
    );

In [0]:
mz11.Single(x => x.VariableType == "ISE11").Value.Should().BeApproximately(lcIa,Precision);
mz12.SingleOrDefault(x => x.VariableType == "ISE11")?.Value.Should().BeApproximately(0,Precision);

In [0]:
mz11.Single(x => x.VariableType == "IFIE1" && x.EstimateType =="C").Value
    .Should().BeApproximately(
        mz12.Single(x => x.VariableType == "IFIE1" && x.EstimateType =="C").Value - lcIa,
        Precision
    );

In [0]:
mz11.Single(x => x.VariableType == "IFIE1" && x.EstimateType == "L").Value.Should().BeApproximately(-lcIa,Precision);
mz12.SingleOrDefault(x => x.VariableType == "IFIE1" && x.EstimateType == "L")?.Value.Should().BeApproximately(0,Precision);